Projeto automação dos resultados por loja

In [42]:
import pandas as pd
import numpy as np
import pathlib
import win32com.client as win32
import pythoncom

In [2]:
#importação da base de dados
emails = pd.read_excel('Emails.xlsx')
lojas = pd.read_excel('lojas.xlsx')
vendas = pd.read_excel('Vendas.xlsx')

In [3]:
#Juntando as tabelas de vendas com as lojas
vendas_lojas = vendas.merge(lojas, on = 'ID Loja', how='inner')

In [4]:
#agrupando as vendas por loja
vendas_grupo = vendas_lojas.groupby('Loja').agg(total_venda = ('Valor Final','sum'))

In [5]:
#criando uma tabela diferente para cada uma das lojas
dicionario_loja = {}
for loja in vendas_lojas['Loja'].values:
    dicionario_loja[loja] = vendas_lojas.loc[vendas_lojas['Loja']==loja,:]

In [6]:
#extraindo o dia para calculo dos indicadores
dia_indicador = vendas['Data'].max()

In [7]:
#fazendo o backup dos arquivos
#verificando as lojas que ja possuem uma pasta na pasta de backup
caminho_backup = pathlib.Path(r'backup')
arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

#criando as pastas de backup de cada loja

for loja in dicionario_loja:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    #salvando o backup dentro da pasta
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month,dia_indicador.day,loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_loja[loja].to_excel(local_arquivo)


# Iremos definir as metas que serão definidas para as lojas

In [16]:
meta_dia_faturamento = 1000
diversidade_produtos_dia_meta = 4
ticket_medio_meta = 500
meta_ano_faturamento_meta = 1650000
diversidade_produtos_ano_meta = 120


# Será criada uma função para calcular os dados que serão mostrados no one page por cada loja para generalizar a solução

In [85]:
def indicadores_one_page(loja, dia):
    """ Função que calcula  os indicadores de cada loja e retorna um dicionario contendo cada 
    indicador como chave e seus resultados como valor.
    
    loja = é um pandas Dataframe referente a loja.
    dia = objeto datetime"""
    faturamento_ano_loja = loja['Valor Final'].sum()
    loja_dia = loja.loc[loja['Data']==dia,:]
    faturamento_loja_dia = loja_dia['Valor Final'].sum()
    diversidade_loja_dia = loja_dia['Produto'].nunique()
    diversidade_loja_ano = loja['Produto'].nunique()
    vendas_loja_grupo = loja.groupby('Código Venda')['Valor Final'].sum().reset_index()
    ticket_medio_loja = vendas_loja_grupo['Valor Final'].mean()
    vendas_loja_grupo_dia = loja_dia.groupby('Código Venda')['Valor Final'].sum().reset_index()
    ticket_medio_loja_dia = vendas_loja_grupo_dia['Valor Final'].mean()
    indicadores_dict = {'faturamento_anual':faturamento_ano_loja,'faturamento_dia':faturamento_loja_dia,
                        'diversidade_produtos_ano':diversidade_loja_ano, 'diversidade_produtos_dia':diversidade_loja_dia,
                        "ticket_medio_ano":ticket_medio_loja,'ticket_medio_dia':ticket_medio_loja_dia}
    return indicadores_dict


# Criando uma Fnção para automatizar o envio de email para o gerente de cada loja

In [87]:
def enviar_email(indicadores,lista_email,nome_loja,dia):
    """ Função para gerar os emails e enviar o relatorio one page junto com o backup em anexo para os gerentes de cada uma das lojas
    indicadores = dicionário contendo cada indicador relacionado a determinada loja
    lista-email = pandas dataframe contendo o nome de cada loja, seu gerente e o email deste.
    nome loja = str contendo o nome da loja
    dia = datetime retirado do dataframe lojas sendo esse o último dia contendo registros.
    """

    #condicionais para definir as cores dos cenarios anuais e diarios
    if indicadores['faturamento_dia']>=meta_dia_faturamento:
        color_dia_fat = 'green'
    else:
        color_dia_fat = 'red'

    if indicadores['diversidade_produtos_dia']>=diversidade_produtos_dia_meta:
        color_dia_div = 'green'
    else:
        color_dia_div = 'red'

    if indicadores['ticket_medio_dia']>=ticket_medio_meta:
        color_tick_dia = 'green'
    else:
        color_tick_dia='red'

    if indicadores['faturamento_anual']>= meta_ano_faturamento_meta:
        color_fat_ano = 'green'
    else:
        color_fat_ano = 'red'

    if indicadores['diversidade_produtos_ano']>= diversidade_produtos_ano_meta:
        color_div_ano = 'green'
    else:
        color_div_ano = 'red'

    if indicadores['ticket_medio_ano']>= ticket_medio_meta:
        color_ticket_ano = 'green'
    else:
        color_ticket_ano = 'red'  


    nome_gerente = lista_email.loc[lista_email['Loja']==nome_loja,'Gerente'].values[0]
    outlook = win32.Dispatch('outlook.application', pythoncom.CoInitialize())
    mail = outlook.CreateItem(0)
    mail.To = f'{lista_email.loc[lista_email['Loja']==nome_loja,'E-mail'].values[0]};denner.souza@partnerpse.com'
    mail.Subject = f'OnePage Dia {dia.day}/{dia.month} - Loja {nome_loja}'
    mail.HTMLBody = f'''
    <p> Bom dia, {nome_gerente} </p>
    <p> O resultado de ontem <strong>({dia.day}/{dia.month})</strong> da <strong>loja {nome_loja}</strong> foi: </p>

    <html>
    <head>
    <style>

    </style>
    </head>
    <body>

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td>{indicadores['faturamento_dia']}</td>
        <td>{meta_dia_faturamento}</td>
        <td><font color = "{color_dia_fat}">◙</font></td>
    </tr>
    <tr>
        <td>Diversidade de produtos</td>
        <td>{indicadores['diversidade_produtos_dia']}</td>
        <td>{diversidade_produtos_dia_meta}</td>
        <td><font color = "{color_dia_div}">◙</font></td>
    </tr>
    <tr>
        <td>Ticket Medio</td>
        <td>{indicadores['ticket_medio_dia']}</td>
        <td>{ticket_medio_meta}</td>
        <td><font color = "{color_tick_dia}">◙</font></td>
    </tr>
    </table>
    </body>
    </html>
    <br>
    <html>
    <head>
    <style>

    </style>
    </head>
    <body>

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Anual </th>
        <th>Meta Anual</th>
        <th>Cenário Anual </th>
    </tr>
    <tr>
        <td>Faturamento Anual </td>
        <td>{indicadores['faturamento_anual']}</td>
        <td>{meta_ano_faturamento_meta}</td>
        <td><font color = "{color_fat_ano}">◙</font></td>
    </tr>
    <tr>
        <td>Diversidade de produtos Anual</td>
        <td>{indicadores['diversidade_produtos_ano']}</td>
        <td>{diversidade_produtos_ano_meta}</td>
        <td><font color = "{color_div_ano}">◙</font></td>
    </tr>
    <tr>
        <td>Ticket Medio Anual </td>
        <td>{indicadores['ticket_medio_ano']}</td>
        <td>{ticket_medio_meta}</td>
        <td><font color = "{color_ticket_ano}">◙</font></td>
    </tr>
    </table>
    </body>
    </html>


    <p> Segue em anexo a planilha com todos os dados para mais detalhes. </p>

    <p> Qualquer dúvida estou a disposição. </p>
    <p> Att, </p>
    '''
    attach = pathlib.Path.cwd()/caminho_backup/nome_loja/f'{dia.month}_{dia.day}_{nome_loja}.xlsx'
    mail.Attachments.Add(str(attach))
    mail.Send()


# Criando uma função que irá gerar os indicadores e enviar os emails na mesma ação, utilizando das duas função criadas anteriormente

In [95]:
def enviar_indicadores(loja_df,dia,lista_email,nome_loja):
    indicador = indicadores_one_page(loja_df,dia)
    enviar_email(indicador,lista_email,nome_loja,dia)

## Criando os indicadores para cada uma das lojas e enviando os emails

In [96]:
for loja in dicionario_loja:
    loja_df = dicionario_loja[loja].copy()
    enviar_indicadores(loja_df,dia_indicador,emails,loja)
   